In [ ]:
import requests
import json
import re
import pandas as pd
import csv
from datetime import datetime

#### ID на новостные группы ВК

In [ ]:
news_id = {
    "life": 24199209,
    "toporcc": 170528132,
    "plohie_novosti_mc": 150709625,
    "trevogznie_novosti": 170191666,
    "the_black_news": 176538220,
    "rentvchannel": 26493942,
    "golos_narodda": 174605620,
    "must_vk": 163822605,
    "nu_i_novosti": 165198246,
    "meduzaproject": 76982440,
    "lentach": 29534144,
    "rgru": 23304496,
    "izvestia": 27532693,
    "tassagency": 26284064,
    "kommersant_ru": 23482909,
    "regnum": 51966513,
    "echomsk": 60556804,
    "forbes_russia_news": 175989174,
    "ria": 15755094,
    "rt_russian": 40316705,
    "vesti": 24136539,
    "1tvnews":49388814,
    "piknvst": 167046442,
    "yznaipervum": 182451980,
    "russiaonliane": 77608619,
    "svezhak_tut": 187796862,
    "ntv": 28658784,
    "5tvpromo": 142991372,
    "russiatv": 24371598,
    "rbc": 25232578,
}

#### ID на вк группы ВУЗов 

In [ ]:
uni_id = {
        "hsemem": 139105204,
        "studlife_hse": 59208488,
        "hse_spb": 299,
        "hse_overheard": 57354358,
        "hseofficial": 132,
        "hse_university": 25205856,
        "msuofmems": 8814453,
        "msu_official": 78019879,
        "overhear_msu": 54295855,
        "msu_info": 227,
        "studsovetmsu": 49962921,
        "fiztehjoke": 24086381,
        "phys_kek": 111557607,
        "miptru": 932,
        "dgapmipt": 26085,
        "miptstream": 26356004,
        "students_of_mipt": 73965718,
        "mfti_overhear": 133687511,
        "fupm_mipt": 1694,
        "urfumemes": 133242721,
        "ural.federal.university": 22941070,
        "student_urfu": 21604404,
        "overhearurfu": 59432260,
        "posnews": 24827886,
        "mephi_official": 69589815,
        "overhearmifi": 59240455,
        "podmephi": 166414649,
        "vmephi": 12742657,
        "bauman_memes": 103133619,
        "bmstu1830": 40427933,
        "lovebmstu": 59336460,
        "studsovet_bmstu": 26724538,
        "baumanka_official": 30539192,
        "baumanka_ru": 107270,
        "bmstu_mt": 18509661,
        "tpunews": 19220040,
        "overhear_tpu": 57308544,
        "itmem": 127149194,
        "itmoru": 94,
        "itmostudents": 2499902,
        "overhearitmo": 58939492,
        "ranepa_overhear": 128219388,
        "rankeks": 146430492, 
        "theacademy": 5398,
        "pa_today": 37389529,
        "mgimo": 26555975,
        "over_hear_mgimo": 57969268,
        "club871": 871,
        "studsouz_mgimo": 25003603,
        "mgimo_reborn": 158818503
        }

#### Сбор данных

In [ ]:
class VkParser():
    
    def __init__(self, domain, owner_id, file, num_posts = 1000):
        self.domain = domain
        self.owner_id = owner_id
        self.file = file
        self.num_posts = num_posts
        self.token = '#######################################################################'
        
        
    def collect_ids(self):
        url = 'https://api.vk.com/method/wall.get'
        offset = 0
        all_ids = []
        while offset < self.num_posts:
            post_ids = []
            requests_params = {
                            'access_token': self.token, 
                            'v': 5.103, 
                            'domain': self.domain,
                            'offset': offset,
                            'count': 100
                                }
            response = requests.get(url, params = requests_params).json()
            try:
                data = response['response']['items']
                for el in data:
                    if datetime.fromtimestamp(el['date']).year == 2020:
                        post_ids.append(el['id'])
                    else:
                        break
                all_ids.extend(post_ids)
            except:
                pass
            offset += 100
        self.ids = all_ids
        return self
    
    
    def write_to_file(self, *args):
        with open(self.file, 'a', newline = '', encoding = 'utf-8') as file:
            writer = csv.writer(file, delimiter = ';')
            writer.writerow(args)
        return self
    
    
    def collect_comments_by_id(self):
        url = 'https://api.vk.com/method/wall.getComments'
        for post_id in self.ids:
            requests_params = {
                        'access_token': self.token, 
                        'v': 5.103, 
                        'owner_id': -self.owner_id,
                        'post_id': post_id,
                        'offset': 0,
                        'count': 100,
                        'thread_items_count': 10,
                        'need_likes': 1
                                }
            response = requests.get(url, params = requests_params).json()
            try:
                comments = response['response']['items']
                for com in comments:
                    likes = com['likes']['count']
                    text = re.sub('[^\w\s.,]', '', com['text'].lower())
                    date = com['date']
                    self.write_to_file(text, likes, date, self.domain)
                    threads = com['thread']['items']
                    for thread in threads:
                        likes = thread['likes']['count']
                        text = re.sub('[^\w\s.,]', '', thread['text'].lower())
                        date = thread['date']
                        self.write_to_file(text, likes, date, self.domain)
            except:
                pass

In [ ]:
for el in uni_id:
    session = VkParser(el, uni_id[el], "mydata.csv", num_posts = 2000)
    session.collect_ids().collect_comments_by_id()